<a href="https://colab.research.google.com/github/vidyasri03/Machine-Learning/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset from an Excel file
data = pd.read_excel('gpt2_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]    # Target variable

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the SVM model
svm = SVC()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]  # Applicable only for polynomial kernel
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit the random search model
random_search.fit(X_train, y_train)

# Print the best parameters found
print("Best Parameters:", random_search.best_params_)

# Get the best estimator
best_svm = random_search.best_estimator_

# Perform cross-validation on the best model
cv_score = cross_validate(best_svm, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)

Best Parameters: {'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'C': 10}
Evaluation Metrics: {'Accuracy Mean': 0.6347, 'Accuracy STD': 0.0322, 'Precision Mean': 0.6457, 'Precision STD': 0.0309, 'Recall Mean': 0.6392, 'Recall STD': 0.035, 'F1 Mean': 0.6397, 'F1 STD': 0.0314}


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE  # For handling class imbalance

# Optional: Scaling the features (this isn't strictly necessary for Decision Trees but may help)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Optional: Handling class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the Decision Tree model
dtree = DecisionTreeClassifier(random_state=42, class_weight='balanced')  # Use class_weight='balanced' to handle class imbalance

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20, 25],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6, 10],
    'max_features': [None, 'sqrt', 'log2'],
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dtree,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=StratifiedKFold(n_splits=10),  # Use StratifiedKFold to preserve class distribution
    scoring='f1_macro',  # F1 macro to handle imbalanced classes
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV
random_search.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

# Get the best Decision Tree model
best_dtree = random_search.best_estimator_

# Perform cross-validation on the best model
cv_score = cross_validate(best_dtree, X_train_resampled, y_train_resampled, cv=StratifiedKFold(n_splits=10),
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)

Best Parameters: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 25, 'criterion': 'gini'}
Evaluation Metrics: {'Accuracy Mean': 0.5874, 'Accuracy STD': 0.0592, 'Precision Mean': 0.5906, 'Precision STD': 0.0586, 'Recall Mean': 0.5874, 'Recall STD': 0.0592, 'F1 Mean': 0.5861, 'F1 STD': 0.0579}


In [ ]:
from sklearn.neural_network import MLPClassifier

# Define the MLP model
mlp = MLPClassifier(max_iter=2000, random_state=42)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'hidden_layer_sizes': [(50,), (100,)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_distributions,
    n_iter=5,
    cv=5,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("MLP Best Parameters:", random_search.best_params_)

# Perform cross-validation on the best model
best_mlp = random_search.best_estimator_
cv_score = cross_validate(best_mlp, X_train, y_train, cv=5, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("MLP Evaluation Metrics:", results)

MLP Best Parameters: {'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Evaluation Metrics: {'Accuracy Mean': 0.622, 'Accuracy STD': 0.0234, 'Precision Mean': 0.626, 'Precision STD': 0.0198, 'Recall Mean': 0.6285, 'Recall STD': 0.0237, 'F1 Mean': 0.6265, 'F1 STD': 0.0212}


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Define the XGBoost model
xgb = XGBClassifier(random_state=42)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01,  0.1],
    'subsample': [ 0.8, 1.0],
    'colsample_bytree': [ 0.8, 1.0],
    'gamma': [0, 1],
    'class_weight': ['balanced']
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_distributions,
    n_iter=5,
    cv=StratifiedKFold(n_splits=5),
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Output best parameters and score
print("Best Parameters:", random_search.best_params_)

# Get the best model
best_xgb = random_search.best_estimator_

# Perform cross-validation on the best XGBoost model
cv_score = cross_validate(best_xgb, X_train, y_train, cv=5,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:41:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 1.0, 'class_weight': 'balanced'}


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:42:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:42:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:43:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:44:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:44:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.



Evaluation Metrics: {'Accuracy Mean': 0.6228, 'Accuracy STD': 0.0166, 'Precision Mean': 0.6302, 'Precision STD': 0.0181, 'Recall Mean': 0.6235, 'Recall STD': 0.0212, 'F1 Mean': 0.6252, 'F1 STD': 0.0183}


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Feature Scaling (important for many models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 2: Handle class imbalance using SMOTE (if needed)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Step 3: Define the AdaBoost model with a shallow decision tree as the base estimator
base_estimator = DecisionTreeClassifier(max_depth=3, random_state=42)

ada = AdaBoostClassifier(
    estimator=base_estimator,
    random_state=42
)

# Step 4: Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],  # Number of boosting rounds (trees)
    'learning_rate': [0.01, 0.1, 1.0],  # How much each estimator contributes
    'estimator__max_depth': [3, 5, 7]  # Max depth of the decision tree (base estimator)
}

# Step 5: Set up RandomizedSearchCV with StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=ada,
    param_distributions=param_distributions,
    n_iter=10,
    cv=skf,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Step 6: Fit the model with the best parameters
random_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters found
print("AdaBoost Best Parameters:", random_search.best_params_)

# Step 7: Perform cross-validation on the best AdaBoost model
best_ada = random_search.best_estimator_
cv_score = cross_validate(
    best_ada, X_train_resampled, y_train_resampled, cv=skf, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False
)
# Step 8: Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("AdaBoost Evaluation Metrics:", results)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Best Parameters: {'n_estimators': 200, 'learning_rate': 1.0, 'estimator__max_depth': 5}


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

AdaBoost Evaluation Metrics: {'Accuracy Mean': 0.6912, 'Accuracy STD': 0.0647, 'Precision Mean': 0.701, 'Precision STD': 0.0567, 'Recall Mean': 0.6912, 'Recall STD': 0.0647, 'F1 Mean': 0.6929, 'F1 STD': 0.0617}


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Feature Scaling (important for many models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 2: Handle class imbalance using SMOTE (if needed)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Step 3: Define the AdaBoost model with a shallow decision tree as the base estimator
base_estimator = DecisionTreeClassifier(max_depth=3, random_state=42)

ada = AdaBoostClassifier(
    estimator=base_estimator,
    random_state=42
)

# Step 4: Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],  # Number of boosting rounds (trees)
    'learning_rate': [0.01, 0.1, 1.0],  # How much each estimator contributes
    'estimator__max_depth': [3, 5, 7]  # Max depth of the decision tree (base estimator)
}

# Step 5: Set up RandomizedSearchCV with StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=ada,
    param_distributions=param_distributions,
    n_iter=10,
    cv=skf,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Step 6: Fit the model with the best parameters
random_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters found
print("AdaBoost Best Parameters:", random_search.best_params_)

# Step 7: Perform cross-validation on the best AdaBoost model
best_ada = random_search.best_estimator_
cv_score = cross_validate(
    best_ada, X_train_resampled, y_train_resampled, cv=skf, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False
)
# Step 8: Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("AdaBoost Evaluation Metrics:", results)

NameError: name 'X_train' is not defined